<a href="https://colab.research.google.com/github/ayselrajabli/KaggleProjects/blob/main/House%20Prices%20-%20Advanced%20Regression%20Techniques/HousePricesKaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
import seaborn as sns
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# Display all columns
pd.set_option('display.max_rows', None)

In [ ]:
url_train = 'https://raw.githubusercontent.com/ayselrajabli/KaggleProjects/main/House%20Prices%20-%20Advanced%20Regression%20Techniques/train.csv'
url_test = 'https://raw.githubusercontent.com/ayselrajabli/KaggleProjects/main/House%20Prices%20-%20Advanced%20Regression%20Techniques/test.csv'

In [ ]:
df_train = pd.read_csv(url_train, encoding = 'unicode_escape')
df_test = pd.read_csv(url_test, encoding = 'unicode_escape')

In [ ]:
df_train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

##EDA

In [ ]:
train_rows, train_cols = df_train.shape
print(f"Train Data: {train_rows} rows \n\t    {train_cols} columns" )
print('----------------------')
test_rows, test_cols = df_test.shape
print(f"Test Data: {test_rows} rows \n\t   {test_cols} columns" )

Train Data: 1460 rows 
	    81 columns
----------------------
Test Data: 1459 rows 
	   80 columns


In [ ]:
#Number of missing values with Percentage
print((df_train.isna().sum()[df_train.isna().sum()!=0]/train_rows*100).round(2).astype(str)+"%")

LotFrontage     17.74%
Alley           93.77%
MasVnrType       0.55%
MasVnrArea       0.55%
BsmtQual         2.53%
BsmtCond         2.53%
BsmtExposure      2.6%
BsmtFinType1     2.53%
BsmtFinType2      2.6%
Electrical       0.07%
FireplaceQu     47.26%
GarageType       5.55%
GarageYrBlt      5.55%
GarageFinish     5.55%
GarageQual       5.55%
GarageCond       5.55%
PoolQC          99.52%
Fence           80.75%
MiscFeature      96.3%
dtype: object


In [ ]:
print((df_test.isna().sum()[df_test.isna().sum()!=0]/train_rows*100).round(2).astype(str)+"%")

MSZoning         0.27%
LotFrontage     15.55%
Alley            92.6%
Utilities        0.14%
Exterior1st      0.07%
Exterior2nd      0.07%
MasVnrType        1.1%
MasVnrArea       1.03%
BsmtQual         3.01%
BsmtCond         3.08%
BsmtExposure     3.01%
BsmtFinType1     2.88%
BsmtFinSF1       0.07%
BsmtFinType2     2.88%
BsmtFinSF2       0.07%
BsmtUnfSF        0.07%
TotalBsmtSF      0.07%
BsmtFullBath     0.14%
BsmtHalfBath     0.14%
KitchenQual      0.07%
Functional       0.14%
FireplaceQu      50.0%
GarageType       5.21%
GarageYrBlt      5.34%
GarageFinish     5.34%
GarageCars       0.07%
GarageArea       0.07%
GarageQual       5.34%
GarageCond       5.34%
PoolQC          99.73%
Fence           80.07%
MiscFeature     96.44%
SaleType         0.07%
dtype: object


In [ ]:
df_train.nunique()

Id               1460
MSSubClass         15
MSZoning            5
LotFrontage       110
LotArea          1073
Street              2
Alley               2
LotShape            4
LandContour         4
Utilities           2
LotConfig           5
LandSlope           3
Neighborhood       25
Condition1          9
Condition2          8
BldgType            5
HouseStyle          8
OverallQual        10
OverallCond         9
YearBuilt         112
YearRemodAdd       61
RoofStyle           6
RoofMatl            8
Exterior1st        15
Exterior2nd        16
MasVnrType          4
MasVnrArea        327
ExterQual           4
ExterCond           5
Foundation          6
BsmtQual            4
BsmtCond            4
BsmtExposure        4
BsmtFinType1        6
BsmtFinSF1        637
BsmtFinType2        6
BsmtFinSF2        144
BsmtUnfSF         780
TotalBsmtSF       721
Heating             6
HeatingQC           5
CentralAir          2
Electrical          5
1stFlrSF          753
2ndFlrSF          417
LowQualFin

In [ ]:
df_test.nunique()

Id               1459
MSSubClass         16
MSZoning            5
LotFrontage       115
LotArea          1106
Street              2
Alley               2
LotShape            4
LandContour         4
Utilities           1
LotConfig           5
LandSlope           3
Neighborhood       25
Condition1          9
Condition2          5
BldgType            5
HouseStyle          7
OverallQual        10
OverallCond         9
YearBuilt         106
YearRemodAdd       61
RoofStyle           6
RoofMatl            4
Exterior1st        13
Exterior2nd        15
MasVnrType          4
MasVnrArea        303
ExterQual           4
ExterCond           5
Foundation          6
BsmtQual            4
BsmtCond            4
BsmtExposure        4
BsmtFinType1        6
BsmtFinSF1        669
BsmtFinType2        6
BsmtFinSF2        161
BsmtUnfSF         793
TotalBsmtSF       736
Heating             4
HeatingQC           5
CentralAir          2
Electrical          4
1stFlrSF          789
2ndFlrSF          407
LowQualFin

In [ ]:
df_train.select_dtypes(exclude=['object']).columns

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [ ]:
corr = ['LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold',]

In [ ]:
for c in corr:
    x = df_train[df_train[c].notna()]
    print(f"Corr. SalePrice ~ {c}: {np.corrcoef(x[c], x['SalePrice'])[0, 1]}")

Corr. SalePrice ~ LotFrontage: 0.35179909657067804
Corr. SalePrice ~ LotArea: 0.2638433538714056
Corr. SalePrice ~ OverallQual: 0.7909816005838044
Corr. SalePrice ~ OverallCond: -0.07785589404867803
Corr. SalePrice ~ YearBuilt: 0.5228973328794969
Corr. SalePrice ~ YearRemodAdd: 0.5071009671113865
Corr. SalePrice ~ MasVnrArea: 0.4774930470957163
Corr. SalePrice ~ BsmtFinSF1: 0.3864198062421532
Corr. SalePrice ~ BsmtFinSF2: -0.011378121450215125
Corr. SalePrice ~ BsmtUnfSF: 0.21447910554696892
Corr. SalePrice ~ TotalBsmtSF: 0.6135805515591956
Corr. SalePrice ~ 1stFlrSF: 0.6058521846919147
Corr. SalePrice ~ 2ndFlrSF: 0.3193338028320681
Corr. SalePrice ~ LowQualFinSF: -0.02560613000067959
Corr. SalePrice ~ GrLivArea: 0.7086244776126523
Corr. SalePrice ~ BsmtFullBath: 0.22712223313149382
Corr. SalePrice ~ BsmtHalfBath: -0.016844154297359016
Corr. SalePrice ~ FullBath: 0.5606637627484449
Corr. SalePrice ~ HalfBath: 0.2841076755947831
Corr. SalePrice ~ BedroomAbvGr: 0.16821315430073988
Corr. 